In [1]:
!pip install -q kokoro>=0.9.2 soundfile
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.12 requires numpy<2,>=1, but you have numpy 2.3.1 which is incompatible.
langchain-community 0.0.28 requires numpy<2,>=1, but you have numpy 2.3.1 which is incompatible.
numba 0.59.1 requires numpy<1.27,>=1.22, but you have numpy 2.3.1 which is incompatible.
scipy 1.14.1 requires numpy<2.3,>=1.23.5, but you have numpy 2.3.1 which is incompatible.
El sistema no puede encontrar la ruta especificada.
El sistema no puede encontrar la ruta especificada.


In [9]:
from kokoro import KPipeline
from IPython.display import display, Audio, HTML
import soundfile as sf
import torch
import os
# Variable que se actualizará
variable_a_modificar = 'Texto a voz'

def get_variable():
    return variable_a_modificar

def set_variable(new_value):
    global variable_a_modificar
    variable_a_modificar = new_value
    print(f"La variable ha sido actualizada a: {variable_a_modificar}")

# Exportar las funciones para que Flask pueda acceder a ellas.  Esto es crucial.
display(HTML("<script>window.getVariable = function() { return " + str(get_variable) + "; };</script>"))
display(HTML("<script>window.setVariable = function(newValue) { " + str(set_variable) + "(newValue); };</script>"))

pipeline = KPipeline(lang_code='f')
text = '''
Je suis un modèle de synthèse vocale léger et efficace, capable de produire des voix naturelles et expressives. Mon architecture optimisée me permet de fonctionner rapidement tout en maintenant une qualité sonore élevée.
Je suis conçu pour être utilisé dans une variété d'applications, allant des assistants virtuels aux jeux vidéo, en passant par les systèmes de navigation et les applications éducatives. Mon objectif est de rendre l'interaction avec la technologie plus humaine et accessible.
'''
# text = '''
# [Kokoro](/kˈOkəɹO/) is an open-weight TTS model with 82 million parameters. Despite its lightweight architecture, it delivers comparable quality to larger models while being significantly faster and more cost-efficient. With Apache-licensed weights, [Kokoro](/kˈOkəɹO/) can be deployed anywhere from production environments to personal projects.
# '''
generator = pipeline(text, voice='af_heart',split_pattern=r'\n+')
for i, (gs, ps, audio) in enumerate(generator):
    print(i, gs, ps)
    display(Audio(data=audio, rate=24000, autoplay=i==0))
    sf.write(f'{i}.wav', audio, 24000)

0 Je suis un modèle de synthèse vocale léger et efficace, capable de produire des voix naturelles et expressives. Mon architecture optimisée me permet de fonctionner rapidement tout en maintenant une qualité sonore élevée. ʒə syˌiz œ̃ modˈɛl də sɛ̃tˈɛz vokˈal leʒˈe e efikˈas, kapˈabl də pʁodyˈiʁ de vwˈa natyʁˈɛlz e ɛkspʁɛsˈiv. mɔ̃n aʁʃitɛktˈyʁ ɔptimizˈe mə pɛʁmˈɛ də fɔ̃ksjɔnˈe ʁapidmˈɑ̃ tut ɑ̃ mɛ̃tnˈɑ̃ yn kalitˈe sonˈɔʁ elvˈe.
0 Je suis un modèle de synthèse vocale léger et efficace, capable de produire des voix naturelles et expressives. Mon architecture optimisée me permet de fonctionner rapidement tout en maintenant une qualité sonore élevée. ʒə syˌiz œ̃ modˈɛl də sɛ̃tˈɛz vokˈal leʒˈe e efikˈas, kapˈabl də pʁodyˈiʁ de vwˈa natyʁˈɛlz e ɛkspʁɛsˈiv. mɔ̃n aʁʃitɛktˈyʁ ɔptimizˈe mə pɛʁmˈɛ də fɔ̃ksjɔnˈe ʁapidmˈɑ̃ tut ɑ̃ mɛ̃tnˈɑ̃ yn kalitˈe sonˈɔʁ elvˈe.


1 Je suis conçu pour être utilisé dans une variété d'applications, allant des assistants virtuels aux jeux vidéo, en passant par les systèmes de navigation et les applications éducatives. Mon objectif est de rendre l'interaction avec la technologie plus humaine et accessible. ʒə syˌi kɔ̃sˈy puʁ ˈɛtʁ ytilizˈe dɑ̃z yn vaʁjetˈe daplikasjˈɔ̃, alˈɑ̃ dez asistˈɑ̃ viʁtyˈɛlz o ʒˈø videˈo, ɑ̃ pasˈɑ̃ paʁ le sistˈɛm də naviɡasjˈɔ̃ e lez aplikasjˈɔ̃z edykatˈiv. mɔ̃n ɔbʒɛktˈif ɛ də ʁˈɑ̃dʁ lɛ̃tɛʁaksjˈɔ̃ avˌɛk la tɛknoloʒˈi plyz ymˈɛn e aksɛsˈibl.


In [ ]:
# Distorsión de voz usando RVC y pydub
import os
from pydub import AudioSegment
from subprocess import run

# --- CONFIGURACIÓN ---
input_audio = "1.wav"                    # Voz original
output_audio = "voz_convertida_rvc.wav"      # Voz convertida sin efectos
final_output = "voz_final_distorsionada.wav" # Voz con efectos

model_path = "weights/mi_modelo.pth"
index_path = "logs/mi_modelo/added_IVF.index"
config_path = "configs/config.json"          # Usa el que corresponda (40k o 48k)

# --- 1. CONVERSIÓN DE VOZ (usa RVC CLI) ---
command = [
    "python", "infer.py", 
    "-m", model_path,
    "-i", input_audio,
    "-o", output_audio,
    "--index", index_path,
    "--f0", "crepe",
    "--pitch", "0",        # Puedes poner -5 o +5 para cambiar el tono
    "--method", "harvest", # harvest o crepe (elige el que mejor te suene)
    "--config", config_path
]
run(command)

# --- 2. DISTORSIÓN POSTPROCESADA ---
# audio = AudioSegment.from_file(output_audio)

# distorted = (
#     audio
#     .low_pass_filter(300)
#     .high_pass_filter(1200)
#     .speedup(playback_speed=1.15)
#     .apply_gain(-3)  # Baja volumen si distorsiona
# )

# distorted.export(final_output, format="wav")
# print(f"✅ Voz distorsionada guardada en: {final_output}")

CompletedProcess(args=['python', 'infer.py', '-m', 'weights/mi_modelo.pth', '-i', 'c:/Users/Marc/Downloads/KC Trabajo/C_TFG_AudioBook/1.wav', '-o', 'voz_convertida_rvc.wav', '--index', 'logs/mi_modelo/added_IVF.index', '--f0', 'crepe', '--pitch', '0', '--method', 'harvest', '--config', 'configs/config.json'], returncode=2)

In [8]:
from IPython.display import display, HTML

display(HTML('''
<div class="container" style="background:rgba(255,255,255,0.95);border-radius:20px;padding:40px;max-width:500px;margin:auto;box-shadow:0 8px 32px rgba(0,0,0,0.1);border:1px solid rgba(255,255,255,0.18);font-family:Segoe UI,Tahoma,Geneva,Verdana,sans-serif;">
    <h1 style="text-align:center;color:#333;margin-bottom:30px;font-size:2.5em;font-weight:300;">🔧 Modificador de Variable</h1>
    <div id="message" style="padding:15px;border-radius:10px;margin-bottom:20px;font-weight:500;text-align:center;display:none;"></div>
    <div class="current-value" style="background:rgba(102,126,234,0.1);border-left:4px solid #667eea;padding:20px;border-radius:10px;margin-bottom:25px;">
        <h3 style="color:#667eea;margin-bottom:10px;font-size:1.2em;">Valor Actual:</h3>
        <div id="currentValue" style="font-size:1.1em;color:#333;font-weight:500;word-wrap:break-word;min-height:25px;">Cargando...</div>
    </div>
    <div class="form-group" style="margin-bottom:25px;">
        <label for="newValue" style="display:block;margin-bottom:8px;color:#555;font-weight:500;font-size:1.1em;">Nuevo Valor:</label>
        <input type="text" id="newValue" placeholder="Introduce el nuevo valor..." style="width:100%;padding:15px;border:2px solid #ddd;border-radius:10px;font-size:1.1em;background:rgba(255,255,255,0.8);">
    </div>
    <div class="buttons" style="display:grid;grid-template-columns:1fr 1fr;gap:15px;margin-bottom:25px;">
        <button onclick="updateVariable()" style="padding:15px 25px;border:none;border-radius:10px;font-size:1.1em;font-weight:600;cursor:pointer;background:linear-gradient(135deg,#667eea,#764ba2);color:white;">Actualizar Variable</button>
        <button onclick="getCurrentValue()" style="padding:15px 25px;border:none;border-radius:10px;font-size:1.1em;font-weight:600;cursor:pointer;background:linear-gradient(135deg,#f093fb,#f5576c);color:white;">Obtener Valor Actual</button>
    </div>
</div>
<script>
function showMessage(text, type = 'success') {
    const messageDiv = document.getElementById('message');
    messageDiv.textContent = text;
    messageDiv.style.display = 'block';
    messageDiv.style.background = type === 'success' ? 'rgba(40,167,69,0.1)' : 'rgba(220,53,69,0.1)';
    messageDiv.style.color = type === 'success' ? '#28a745' : '#dc3545';
    setTimeout(() => { messageDiv.style.display = 'none'; }, 3000);
}
async function getCurrentValue() {
    try {
        const response = await fetch('http://localhost:5000/api/variable');
        const data = await response.json();
        if (data.status === 'success') {
            document.getElementById('currentValue').textContent = data.value || 'Sin valor';
            showMessage('Valor obtenido correctamente', 'success');
        } else {
            showMessage('Error al obtener el valor', 'error');
        }
    } catch (error) {
        showMessage('Error de conexión', 'error');
    }
}
async function updateVariable() {
    const newValue = document.getElementById('newValue').value;
    if (!newValue.trim()) {
        showMessage('Por favor introduce un valor', 'error');
        return;
    }
    try {
        const response = await fetch('http://localhost:5000/api/variable', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ value: newValue })
        });
        const data = await response.json();
        if (data.status === 'success') {
            document.getElementById('currentValue').textContent = data.value;
            document.getElementById('newValue').value = '';
            showMessage(data.message, 'success');
        } else {
            showMessage(data.message || 'Error al actualizar', 'error');
        }
    } catch (error) {
        showMessage('Error de conexión', 'error');
    }
}
document.getElementById('newValue').addEventListener('keypress', function(e) {
    if (e.key === 'Enter') { updateVariable(); }
});
document.addEventListener('DOMContentLoaded', function() { getCurrentValue(); });
getCurrentValue();
</script>
'''))